In [1]:
# Imports
from Code.h5Constructor import h5Constructor

In [2]:
gc = h5Constructor('./Dataset/CIFs/Val/', './Dataset/h5/Val/')

In [3]:
gc.gen_h5s(parallelize=False)


Constructing graphs from cif files:


  0%|          | 0/7 [00:00<?, ?it/s]





















 14%|█▍        | 1/7 [00:44<04:26, 44.44s/it]




















 29%|██▊       | 2/7 [01:20<03:18, 39.68s/it]


















 43%|████▎     | 3/7 [01:46<02:12, 33.22s/it]

















 57%|█████▋    | 4/7 [02:11<01:30, 30.10s/it]



















 71%|███████▏  | 5/7 [02:41<01:00, 30.07s/it]










 86%|████████▌ | 6/7 [02:58<00:25, 25.55s/it]























100%|██████████| 7/7 [03:41<00:00, 31.61s/it]


In [ ]:
from Code.datasetClass import InOrgMatDatasets

test = InOrgMatDatasets('DatasetTest', root='./Dataset/')

Processing...
Done!


In [ ]:
test.raw_file_names

['Train/ReO3_CoO3.h5',
 'Train/AntiFlourite_Ir2O.h5',
 'Train/NaCl_FeO.h5',
 'Train/Flourite_MoO2.h5',
 'Train/Rutile_MoO2.h5',
 'Train/NaCl_CoO.h5',
 'Train/Flourite_IrO2.h5',
 'Train/CdI2_IrO2.h5',
 'Train/CdI2_FeO2.h5',
 'Train/ReO3_IrO3.h5',
 'Train/Rutile_CoO2.h5',
 'Train/NaCl_IrO.h5',
 'Train/Spinel_CoCo2O4.h5',
 'Train/ReO3_MoO3.h5',
 'Train/AntiFlourite_Fe2O.h5',
 'Train/Spinel_MoMo2O4.h5',
 'Train/CdI2_CoO2.h5',
 'Train/Spinel_FeFe2O4.h5',
 'Train/AntiFlourite_Co2O.h5',
 'Train/ReO3_FeO3.h5',
 'Train/Flourite_FeO2.h5',
 'Train/AntiFlourite_Mo2O.h5',
 'Train/CdI2_MoO2.h5',
 'Train/Flourite_CoO2.h5',
 'Train/Rutile_FeO2.h5',
 'Train/Spinel_IrIr2O4.h5',
 'Train/Rutile_IrO2.h5',
 'Train/NaCl_MoO.h5']

In [ ]:
test.get(100)

Data(
  x=[12, 4],
  edge_index=[2, 64],
  edge_attr=[88],
  y={
    np_size=11.35,
    nd=[580],
    xrd=[580],
    nPDF=[301],
    xPDF=[301],
    sans=[300],
    saxs=[300]
  }
)